## Intended for `voila-gridstack`
Meaning that the output layouts are optimized for the [voila](https://github.com/voila-dashboards/voila-gridstack) interface. 

#### You can still execute the cells sequentially to interact with inline outputs.

In [ ]:
from __future__ import print_function
from scipy.io import loadmat
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import joypy
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import feather
from IPython.display import display
import warnings
warnings.filterwarnings("ignore")
import os
filesep = os.path.sep

For each subject, you can switch between different histograms (`original`,`modzscore`,`modzscore_reduced`) to compare a selected muscle part for the respective shift function. You can isolate C<sup>α=σ</sup> combinations for easier comparison with their HSF counterparts.

You can isolate shift functions for a muscle part (double click in the legend) and hover over plotted objects to see corresponding values.

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# PLEASE PROVIDE Stats_derivatives folder path if you are executing this notebook locally
datDir = '/tmp/Stats_derivatives'

In [ ]:
def vectorize_mat(filename,region):
    mat = loadmat(filename)
    tmp = mat[region]
    tmp = tmp['scalars']
    tmp = np.squeeze(tmp[:,0][0])
    vec = tmp.tolist()
    return vec

def getTableParts(sub,regn,trsf):
    idx = [4,6,8,10]
    aa = []
    bb=[]
    # Follow sigma first order for consistency between figures
    # Sigma |4|6|8|10|4|6|8|10|...
    # Alpha |4|4|4|4 |6|6|6|6 |...
    for jj in idx:
        for ii in idx: 
            if trsf == 'modzscore':
                curVec = vectorize_mat(datDir + filesep + 'FiberStrain_mzscore'+ filesep + 'sub' + sub + '_sigma' + str(ii) + '_alpha' + str(jj) + '_modzscore.mat',regn)
            elif trsf == 'orig':  
                curVec = vectorize_mat(datDir + filesep + 'FiberStrain_parts' + filesep +  'sub' + sub + '_sigma' + str(ii) + '_alpha' + str(jj) + '_seg10.mat',regn)
            elif trsf == 'reduced':
                curVec = vectorize_mat(datDir + filesep + 'FiberStrain_mzscore_reduced'+ filesep + 'sub' + sub + '_sigma' + str(ii) + '_alpha' + str(jj) + '_modzscore_reduced.mat',regn)
            aa.append(curVec)
            # Enforce consistent alphabetical order
            if ii!=10 and jj!=10:
                bb.append(['alpha0' + str(jj) + '_sigma0' + str(ii)]*len(curVec))
            elif ii==10 and jj!=10:    
                bb.append(['alpha0' + str(jj) + '_sigma' + str(ii)]*len(curVec))
            elif ii!=10 and jj==10:    
                bb.append(['alpha' + str(jj) + '_sigma0' + str(ii)]*len(curVec))
            else:
                 bb.append(['alpha' + str(jj) + '_sigma' + str(ii)]*len(curVec))
                    
    aa = np.array(aa)
    bb = np.array(bb)    
    aa = aa.flatten()
    bb = bb.flatten()
    dat = {'str':aa,'lbl':bb}
    df = pd.DataFrame(data=dat)
    return df

def getTablePartsEqual(sub,regn,trsf,idx):
    aa = []
    bb=[]

    for ii in idx:
        if trsf == 'modzscore':
            curVec = vectorize_mat(datDir + filesep + 'FiberStrain_mzscore'+ filesep + 'sub' + sub + '_sigma' + str(ii) + '_alpha' + str(ii) + '_modzscore.mat',regn)
        elif trsf == 'orig':  
            curVec = vectorize_mat(datDir + filesep  + 'FiberStrain_parts' + filesep  + 'sub' + sub + '_sigma' + str(ii) + '_alpha' + str(ii) + '_seg10.mat',regn)
        elif trsf == 'reduced':
            curVec = vectorize_mat(datDir + filesep + 'FiberStrain_mzscore_reduced'+ filesep + 'sub' + sub + '_sigma' + str(ii) + '_alpha' + str(ii) + '_modzscore_reduced.mat',regn)
        # Enforce consistent order
        aa.append(curVec) 
        if ii!=10:
            bb.append(['sigma0' + str(ii) + '_alpha0' + str(ii)]*len(curVec))
        else:
            bb.append(['sigma' + str(ii) + '_alpha' + str(ii)]*len(curVec))
        
 
    aa = np.array(aa)
    bb = np.array(bb)    
    aa = aa.flatten()
    bb = bb.flatten()
    dat = {'str':aa,'lbl':bb}
    df = pd.DataFrame(data=dat)
    return df

%matplotlib inline
def on_change(Subject,Segment,Equals,C_44,C_66,C_88,C_1010,section):
    # If equal combinations are selected, further allow user to 
    # display a selection. Collect their idx here based on the 
    # checkbox vals
    if Equals:
        selectC = [{'4':C_44},{'6':C_66},{'8':C_88},{'10':C_1010}]
        eqIdxs = []
        for c in selectC:
            for key, value in c.items():
                if c[key]:
                     eqIdxs.append(int(key))
    else:
        eqIdxs = []
    
    # Sync colors with those in the HSF grid                
    colorPairs = {'p4':'#762a83','p3':'#9970ab','p2':'#c2a5cf','p1':'#e7d4e8','m1':'#e08214','m2':'#fdb863','d1':'#e6f5d0','d2':'#b8e186','d3':'#7fbc41','d4':'#4d9221'}
    
    if section=='original':
        # Show original histogramgs
        if not Equals:
            df1 =getTableParts(Subject,Segment,'orig')
        else:
            df1 =getTablePartsEqual(Subject,Segment,'orig',eqIdxs)
        fig, ax = joypy.joyplot(df1, by="lbl", column="str", range_style='own', 
                              grid='y', linewidth=1, legend=False, fade=True, figsize=(9,8),
                              title=Subject+ ' ' + Segment + ' Original',
                              kind="kde", bins=20,colormap=ListedColormap([colorPairs[Segment]]),background='w',ylabelsize=8)
    if section=='mzscore':
        # Show modified z-scored histograms 
        if not Equals:
            df2 =getTableParts(Subject,Segment,'modzscore')
        else:
            df2 =getTablePartsEqual(Subject,Segment,'modzscore',eqIdxs)
        fig, ax = joypy.joyplot(df2, by="lbl", column="str", range_style='own', grid='y', 
                              linewidth=1, legend=False, fade=True, figsize=(9,8),
                              title=Subject+ ' ' + Segment + ' Mod. z-score',
                              kind="kde", bins=20,colormap=ListedColormap([colorPairs[Segment]]),background='w', ylabelsize=8)
    if section=='mzscore-reduced':
        # Show histograms after uniform (spatial) downsampling of modified z-scores
        if not Equals:
            df3 =getTableParts(Subject,Segment,'reduced')
        else:
            df3 =getTablePartsEqual(Subject,Segment,'reduced',eqIdxs)
        df2 =getTableParts(Subject,Segment,'reduced')
        fig, ax = joypy.joyplot(df3, by="lbl", column="str", range_style='own', 
                              grid="y", linewidth=1, legend=False, fade=True, figsize=(9,8),
                              title=Subject+ ' ' + Segment + ' Mod. z-score downsampled',
                              kind="kde", bins=20,colormap=ListedColormap([colorPairs[Segment]]),background='w', ylabelsize=8)
        plt.show()

In [ ]:
def getUI():
    subject = widgets.Dropdown(
        options=['A', 'B', 'C','D','E'],
        value='A',
        description='Subject:',
        disabled=False,
        width = 100)
    part = widgets.Dropdown(
        options=['p4','p3','p2','p1','m1','m2','d1','d2','d3','d4'],
        value='p4',
        description='Muscle part:',
        disabled=False,
        width = 100)
    buttons = widgets.ToggleButtons(
        options=['original', 'mzscore', 'mzscore-reduced'],
        description='Speed:',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=['Original distributions', 'Distributions after modified z-score standardization', 'Uniform down-sampled modified z-score distributions'],
        layout = widgets.Layout(width='100%'))
    equals = widgets.Checkbox(
        value=False,
        description='<b>C<sup>α=σ</sup></b>',
        disabled=False,
        indent=False
    )
    c_44 = widgets.Checkbox(
        value=False,
        description='C<sup><4,4></sup>',
        disabled=False,
        indent=False
    )
    c_66 = widgets.Checkbox(
        value=False,
        description='C<sup><6,6></sup>',
        disabled=False,
        indent=False
    )
    c_88 = widgets.Checkbox(
        value=False,
        description='C<sup><8,8></sup>',
        disabled=False,
        indent=False
    )
    c_1010 = widgets.Checkbox(
        value=False,
        description='C<sup><10,10></sup>',
        disabled=False,
        indent=False
    )
    layout=widgets.Layout(width='90%')
    layout2=widgets.Layout(flex='1 1 auto',width='80%',margin='0% 0% 0% 10%')
    ui = widgets.VBox([widgets.HBox([subject, part],layout=layout),buttons,widgets.HBox([equals,c_44,c_66,c_88,c_1010],layout=layout2)])
    out = widgets.interactive_output(on_change, {'Subject': subject, 'Segment': part, 'Equals': equals,'C_44':c_44,'C_66':c_66,'C_88':c_88,'C_1010':c_1010,'section':buttons})
    return ui, out

aa,bb=getUI()
display(aa,bb)


## IMPORTANT NOTE

The interactive figure visualized below (`HSF_GRID.html`) was generated by the `step3_runHSF_and_visualize.ipynb` notebook to enable side by side comparison of the HSF results with the histograms on the voila dashboard. 

* If you don't have HSF_GRID.html in the same folder with this notebook, please run `step3` notebook to generate one. 

In [ ]:
file = './HSF_GRID.html'
from IPython.display import IFrame
IFrame(src=file, width=1100, height=1100)